# Tutorial: PyTorch to CasADi NMPC via ONNX
## Part 2: Importing ONNX Model into CasADi and NMPC Formulation

In Part 1, we successfully trained a PyTorch neural network to model the Van der Pol oscillator's dynamics and exported it to an ONNX file (`vdp_ann_model.onnx`).

Now, in Part 2, we will:
1.  Import this ONNX model into **CasADi**, creating a symbolic CasADi `Function`.
2.  Develop a wrapper around this CasADi function to handle the necessary data scaling and unscaling (consistent with how the PyTorch model was trained).
3.  Use this wrapped CasADi function as the dynamic model within a Nonlinear Model Predictive Control (NMPC) problem formulation using CasADi's `Opti()` stack.
4.  Define the NMPC objective function (e.g., setpoint tracking) and constraints.
5.  Configure an NLP solver (like IPOPT) for the NMPC problem.

**Prerequisites:** Completion of Part 1, basic understanding of CasADi and NMPC principles.

### 2.1 Importing Libraries and Loading Scalers

We need NumPy, Matplotlib, CasADi, and importantly, we need to load the `input_feature_scaler` and `output_target_scaler` that were `fit` in Part 1. These scalers are crucial for correct inference with the ONNX model.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import casadi as ca
import onnx # To inspect ONNX model if needed, though CasADi handles import
import onnxruntime # For potential direct ONNX verification if CasADi import is tricky
from sklearn.preprocessing import MinMaxScaler # To define scalers (assuming they weren't saved)
import joblib # For saving/loading sklearn scalers

plt.rcParams.update({'font.size': 12, 'figure.figsize': (10, 6)}) 

# --- Attempt to load scalers if saved from Part 1 ---
# In a real workflow, you would save these from Part 1, e.g., using joblib
# joblib.dump(input_feature_scaler, 'input_feature_scaler.pkl')
# joblib.dump(output_target_scaler, 'output_target_scaler.pkl')
try:
    input_feature_scaler = joblib.load('input_feature_scaler.pkl')
    output_target_scaler = joblib.load('output_target_scaler.pkl')
    print("Loaded scalers from .pkl files.")
except FileNotFoundError:
    print("Scaler .pkl files not found. ")
    print("Please ensure you run Part 1 to generate and save them, or redefine them here based on Part 1 data.")
    # For this notebook to run standalone if Part 1's environment isn't directly carried over,
    # we might need to re-generate some dummy data just to fit scalers with expected dimensions.
    # This is NOT ideal but a fallback for a standalone tutorial part.
    print("Attempting to create placeholder scalers based on expected dimensions from Part 1...")
    n_x_vdp_part1 = 2
    n_u_vdp_part1 = 1
    # Create dummy data with plausible ranges for Van der Pol (x1, x2, u)
    dummy_input_features = np.random.uniform(low=[-3, -3, -2], high=[3, 3, 2], size=(100, n_x_vdp_part1 + n_u_vdp_part1))
    dummy_output_targets = np.random.uniform(low=[-3, -3], high=[3, 3], size=(100, n_x_vdp_part1))
    input_feature_scaler = MinMaxScaler(feature_range=(-1, 1))
    output_target_scaler = MinMaxScaler(feature_range=(-1, 1))
    input_feature_scaler.fit(dummy_input_features)
    output_target_scaler.fit(dummy_output_targets)
    print("Placeholder scalers created and fitted with dummy data. For best results, use scalers from Part 1.")

# Path to the ONNX model exported from Part 1
onnx_model_filepath = "vdp_ann_model.onnx"

# System dimensions (should match Part 1)
n_x = 2 # number of states (Ca, T for CSTR, or x1, x2 for VDP)
n_u = 1 # number of inputs (Tc for CSTR, or u for VDP)
ann_input_dim_part2 = n_x + n_u
ann_output_dim_part2 = n_x

Scaler .pkl files not found. 
Please ensure you run Part 1 to generate and save them, or redefine them here based on Part 1 data.
Attempting to create placeholder scalers based on expected dimensions from Part 1...
Placeholder scalers created and fitted with dummy data. For best results, use scalers from Part 1.


### 2.2 Importing the ONNX Model into CasADi

CasADi provides the `ca.Function.external` interface or the `ca.Importer` class to load models from various formats, including ONNX. We will use `ca.external` as it's often more straightforward if your ONNX model has clearly named inputs and outputs (which we defined during export in Part 1).

The key is that CasADi will treat this imported ONNX model as a black-box function initially, but it can perform AD *through* it if the operations within the ONNX graph are supported by CasADi's ONNX AD capabilities.

In [2]:
try:
    # Create a CasADi function from the ONNX model
    # The first argument to ca.external is the desired name of the CasADi function.
    # The second is the path to the ONNX file.
    # CasADi attempts to infer input/output names and dimensions from the ONNX file.
    # These names should match what we specified during torch.onnx.export.
    onnx_casadi_func_scaled = ca.external('f_onnx_scaled', onnx_model_filepath)
    print("ONNX model loaded into CasADi function successfully.")
    
    # Let's inspect the function's signature to be sure
    print(f"CasADi ONNX function input names: {onnx_casadi_func_scaled.name_in()}")
    print(f"CasADi ONNX function output names: {onnx_casadi_func_scaled.name_out()}")
    print(f"CasADi ONNX function input sizes: {onnx_casadi_func_scaled.sz_in()}")
    print(f"CasADi ONNX function output sizes: {onnx_casadi_func_scaled.sz_out()}")

    # Expected: Input name 'input_scaled_features', size (ann_input_dim_part2, 1)
    #           Output name 'output_scaled_next_state', size (ann_output_dim_part2, 1)
    assert onnx_casadi_func_scaled.name_in()[0] == "input_scaled_features", "Input name mismatch!"
    assert onnx_casadi_func_scaled.name_out()[0] == "output_scaled_next_state", "Output name mismatch!"
    assert onnx_casadi_func_scaled.sz_in()[0] == (ann_input_dim_part2, 1), "Input size mismatch!"
    assert onnx_casadi_func_scaled.sz_out()[0] == (ann_output_dim_part2, 1), "Output size mismatch!"

except Exception as e:
    print(f"Error loading ONNX model into CasADi: {e}")
    print("Ensure 'vdp_ann_model.onnx' exists and CasADi's ONNX interface is functional.")
    onnx_casadi_func_scaled = None # Set to None if import fails

Error loading ONNX model into CasADi: .../casadi/core/casadi_os.cpp:166: Assertion "handle!=nullptr" failed:
DllLibrary::init_handle: Cannot load shared library 'vdp_ann_model.onnx': 
   (
    Searched directories: 1. casadipath from GlobalOptions
                          2. CASADIPATH env var
                          3. PATH env var (Windows)
                          4. LD_LIBRARY_PATH env var (Linux)
                          5. DYLD_LIBRARY_PATH env var (osx)
    A library may be 'not found' even if the file exists:
          * library is not compatible (different compiler/bitness)
          * the dependencies are not found
   )
  Tried '/home/tensor/Model_predictive_control/.venv/lib/python3.12/site-packages/casadi' :
    Error code: /home/tensor/Model_predictive_control/.venv/lib/python3.12/site-packages/casadi/vdp_ann_model.onnx: cannot open shared object file: No such file or directory
  Tried '' :
    Error code: vdp_ann_model.onnx: cannot open shared object file: No such fi

### 2.3 Creating a CasADi Wrapper for Scaling and Unscaling

The `onnx_casadi_func_scaled` expects scaled inputs and produces scaled outputs. For our NMPC, the states $x_k$ and inputs $u_k$ will typically be in their original physical units. So, we need a wrapper CasADi function that handles the scaling and unscaling internally.

**Scaling:** $X_{scaled} = (X_{unscaled} - X_{min}) / (X_{max} - X_{min}) = X_{unscaled} \cdot \text{scale_} + \text{offset_}$
where `scale_` is `1 / (data_max_ - data_min_)` and `offset_` is `-data_min_ * scale_` from `MinMaxScaler`. (Or `(X - mean) / std` for `StandardScaler`).

**Unscaling:** $X_{unscaled} = X_{scaled} / \text{scale_} - \text{offset_} / \text{scale_} = X_{scaled} / \text{scale_} + X_{min}$

The scaler parameters (`min_`, `scale_` for `MinMaxScaler`) must be treated as numerical constants within this CasADi wrapper.

In [3]:
if onnx_casadi_func_scaled is not None:
    # Extract scaler parameters (these are NumPy arrays)
    input_min_np = input_feature_scaler.min_
    input_scale_np = input_feature_scaler.scale_ # This is 1 / (max - min)
    output_min_np = output_target_scaler.min_
    output_scale_np = output_target_scaler.scale_

    # Define symbolic inputs for the wrapper function (unscaled)
    sx_k_unscaled = ca.SX.sym('sx_k_unscaled', n_x)
    su_k_unscaled = ca.SX.sym('su_k_unscaled', n_u)

    # 1. Combine and scale inputs for the ONNX model
    s_ann_input_unscaled = ca.vertcat(sx_k_unscaled, su_k_unscaled)
    s_ann_input_scaled = (s_ann_input_unscaled - input_min_np.reshape(-1,1)) * input_scale_np.reshape(-1,1)

    # 2. Call the imported scaled ONNX function
    s_ann_output_scaled = onnx_casadi_func_scaled(s_ann_input_scaled)

    # 3. Unscale the output of the ONNX model
    # output_unscaled = output_scaled / scale_ + min_ (since scale_ is 1/(max-min))
    sx_k_plus_1_unscaled = s_ann_output_scaled / output_scale_np.reshape(-1,1) + output_min_np.reshape(-1,1)

    # Create the final CasADi function f_ann_mpc(xk, uk) -> xk+1
    f_ann_mpc_casadi = ca.Function('f_ann_mpc', 
                                   [sx_k_unscaled, su_k_unscaled], 
                                   [sx_k_plus_1_unscaled],
                                   ['xk', 'uk'], ['xk_plus_1'])
    print("CasADi wrapper function 'f_ann_mpc_casadi' (unscaled in/out) created.")

    # Test the wrapper with an example
    test_x_np = np.array([0.5, 0.1])
    test_u_np = np.array([0.2])
    
    pred_next_state_casadi = f_ann_mpc_casadi(test_x_np, test_u_np)
    print(f"Test input x_k: {test_x_np}, u_k: {test_u_np}")
    print(f"Predicted x_k+1 from CasADi wrapper: {pred_next_state_casadi}")

    # Compare with manual PyTorch + scaling for verification
    ann_input_manual_unscaled = np.concatenate((test_x_np, test_u_np)).reshape(1,-1)
    ann_input_manual_scaled = input_feature_scaler.transform(ann_input_manual_unscaled)
    with torch.no_grad():
        vdp_ann_model.eval()
        pytorch_output_manual_scaled = vdp_ann_model(torch.tensor(ann_input_manual_scaled, dtype=torch.float32))
        pytorch_output_manual_unscaled = output_target_scaler.inverse_transform(pytorch_output_manual_scaled.numpy())
    print(f"Predicted x_k+1 from PyTorch (manual scaling): {pytorch_output_manual_unscaled.flatten()}")
    assert np.allclose(np.array(pred_next_state_casadi).flatten(), pytorch_output_manual_unscaled.flatten(), atol=1e-5),
           "CasADi wrapper output does not match manual PyTorch + scaling!"
    print("CasADi wrapper matches manual PyTorch + scaling test.")
else:
    print("Skipping CasADi wrapper creation as ONNX import failed.")
    f_ann_mpc_casadi = None

SyntaxError: invalid syntax (179902542.py, line 46)

### 2.4 NMPC Problem Formulation

Now we set up the NMPC optimization problem using `Opti()` and our `f_ann_mpc_casadi` function for the system dynamics.

In [ ]:
if f_ann_mpc_casadi:
    # NMPC Parameters for Van der Pol
    Ts_mpc_vdp = Ts_vdp_data # Control interval
    Np_vdp = 15          # Prediction horizon

    # Objective Function Weights
    Q_x1_vdp = 10.0  # Weight for x1 tracking error
    Q_x2_vdp = 1.0   # Weight for x2 tracking error (optional)
    R_u_vdp = 0.1    # Weight for u (control input) magnitude
    S_u_vdp = 0.5    # Weight for delta_u (control input rate)

    # Constraints
    u_min_vdp = -2.0; u_max_vdp = 2.0
    delta_u_max_vdp = 0.5 # per Ts_mpc_vdp
    x1_min_vdp = -2.5; x1_max_vdp = 2.5
    x2_min_vdp = -3.0; x2_max_vdp = 3.0

    # Setpoints 
    x1_sp_target_vdp = 0.0 
    x2_sp_target_vdp = 0.0 # Target origin (stabilization)

    opti_vdp = ca.Opti()

    # Decision variables
    X_sym_vdp = opti_vdp.variable(n_x, Np_vdp + 1) # States x_k to x_k+Np
    U_sym_vdp = opti_vdp.variable(n_u, Np_vdp)   # Inputs u_k to u_k+Np-1

    # Parameters
    x0_vdp_param = opti_vdp.parameter(n_x)
    u_prev_vdp_param = opti_vdp.parameter(n_u)
    x1_sp_vdp_param = opti_vdp.parameter(Np_vdp) # x1 setpoint trajectory for x_k+1 to x_k+Np
    x2_sp_vdp_param = opti_vdp.parameter(Np_vdp) # x2 setpoint trajectory

    # Objective function
    obj_vdp = 0
    for j in range(Np_vdp):
        obj_vdp += Q_x1_vdp * (X_sym_vdp[0, j+1] - x1_sp_vdp_param[j])**2
        obj_vdp += Q_x2_vdp * (X_sym_vdp[1, j+1] - x2_sp_vdp_param[j])**2
        obj_vdp += R_u_vdp * (U_sym_vdp[0, j])**2
        delta_u_vdp = U_sym_vdp[0, j] - (u_prev_vdp_param[0] if j==0 else U_sym_vdp[0, j-1])
        obj_vdp += S_u_vdp * delta_u_vdp**2
    opti_vdp.minimize(obj_vdp)

    # Dynamic constraints using the f_ann_mpc_casadi wrapper
    opti_vdp.subject_to(X_sym_vdp[:,0] == x0_vdp_param)
    for j in range(Np_vdp):
        x_next_pred_vdp = f_ann_mpc_casadi(X_sym_vdp[:,j], U_sym_vdp[:,j])
        opti_vdp.subject_to(X_sym_vdp[:,j+1] == x_next_pred_vdp)

    # Path and input constraints
    for j in range(Np_vdp):
        opti_vdp.subject_to(opti_vdp.bounded(u_min_vdp, U_sym_vdp[0,j], u_max_vdp))
        delta_u_constr_vdp = U_sym_vdp[0,j] - (u_prev_vdp_param[0] if j==0 else U_sym_vdp[0,j-1])
        opti_vdp.subject_to(opti_vdp.bounded(-delta_u_max_vdp, delta_u_constr_vdp, delta_u_max_vdp))
        
        opti_vdp.subject_to(opti_vdp.bounded(x1_min_vdp, X_sym_vdp[0,j+1], x1_max_vdp))
        opti_vdp.subject_to(opti_vdp.bounded(x2_min_vdp, X_sym_vdp[1,j+1], x2_max_vdp))

    # NLP solver options
    nlp_opts_vdp = {'ipopt.print_level': 0, 'print_time': 0, 'ipopt.max_iter': 100,
                    'ipopt.acceptable_tol': 1e-6, 'ipopt.acceptable_obj_change_tol': 1e-6}
    opti_vdp.solver('ipopt', nlp_opts_vdp)
    print("Van der Pol NMPC problem (using ONNX-ANN model) formulated.")
else:
    print("Skipping NMPC formulation as CasADi ONNX wrapper was not created.")

## Summary of Part 2

In this part, we have:
1.  Successfully imported the ONNX model (exported from PyTorch in Part 1) into CasADi, creating a symbolic `Function`.
2.  Verified the input/output structure of this imported function.
3.  Created a critical CasADi wrapper function (`f_ann_mpc_casadi`) that encapsulates the ONNX model call along with the necessary data scaling and unscaling operations. This wrapper presents a clean $x_{k+1} = f(x_k, u_k)$ interface using unscaled physical units.
4.  Formulated a complete NMPC optimization problem using CasADi's `Opti()` stack, employing our `f_ann_mpc_casadi` wrapper as the dynamic model for prediction.
5.  Defined an objective function for setpoint tracking and included relevant input and state constraints.
6.  Configured IPOPT as the NLP solver for this NMPC problem.

**Next Steps (Part 3 of this Tutorial):**
We will implement the receding horizon control loop, simulate the closed-loop ANN-NMPC system controlling the "true" Van der Pol oscillator, and analyze its performance.